# TKO_2096 Applications of Data Analysis 2021
## Exercise 4

Complete the tasks given to you in the letter below. There are cells at the end of this notebook to which you are expected to write your code. Insert markdown cells as needed to describe your solution.

The deadline of this exercise is **28.2.2021, 23:59 PM**. Please contact Juho Heimonen (juaheim@utu.fi) if you have any questions about this exercise.


---

Student name: Tomi Salomaa

Student number: 

Student email: 

---


Dear Data Scientist,

I have a task for you that concerns drug molecules and their targets. I have spent a lot of time in a laboratory to measure how strongly potential drug molecules bind to putative target molecules. I do not have enough resources to measure all possible drug-target pairs, so I would like to first predict their affinities and then measure only the most promising ones. I have already managed to create a model which I believe is good for this purpose. Its details are below.

- algorithm: K-nearest neighbours regressor
- parameters: K=20
- training data: full data set

The full data set is available as the files `input.data`, `output.data` and `pairs.data` for you to use. The first file contains the features of pairs, whereas the second contains their affinities. The third file contains the identifiers of the drug and target molecules of which the pairs are composed. The files are paired, i.e. the i<sup>*th*</sup> row in each file is about the same pair.

I am not able to evaluate how well my model will perform when I will use it to predict the affinities of new drug-target pairs. I need you to evaluate the model for me. There are three distinct situations in which I want to use this model in the future.

1. I did not have the resources to measure the affinities of all the known drug-target pairs in the laboratory, so I want to use the model to predict the affinities of the remaining pairs.
2. I am confident that I will discover new potential drug molecules in the future, so I will want to use the model to predict their affinities to the currently known targets.
3. Because new putative target molecules, too, will likely be identified in the future, I will also want to use the model to predict the affinities between the drug molecules I will discover and the target molecules somebody else will discover in the future.

I need to get evaluation results from leave-one-out cross-validation with C-index. Please evaluate the generalisation performance of my model in the three situations and explain why your cross-validation methods are suitable for them.


Yours sincerely, \
Bio Scientist


PS. Follow all the general exercise guidelines stated in Moodle.

---

#### Import libraries

In [1]:
# Import the libraries you need.
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.spatial import distance
from google.colab import files
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler

#### Load datasets

In [2]:
# Uploading for google colab environment
uploaded = files.upload()

Saving pairs.data to pairs.data
Saving output.data to output.data
Saving input.data to input.data


In [3]:
# Read the data files (input.data, output.data, pairs.data).
# Using delimiter to assign separate values into columns.
input_data = pd.read_csv('input.data',header=None,delimiter=' ')
output_data = pd.read_csv('output.data',header=None,delimiter=' ')
pairs_data = pd.read_csv('pairs.data',header=None,delimiter=' ')

In [4]:
# Sneak peek to what's inside (aka searching for nulls and learning dimensions)
print('*'*35)
print('INPUT DATA INFO')
print('*'*35)
print(input_data.info())
print('\nSample:',input_data.head())
print('\n')
print('*'*35)
print('OUTPUT DATA INFO')
print('*'*35)
print(output_data.info())
print('\nSample:',output_data.head())
print('\n')
print('*'*35)
print('PAIRS DATA INFO')
print('*'*35)
print(pairs_data.info())
print('\nSample:',pairs_data.head())

***********************************
INPUT DATA INFO
***********************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Columns: 2500 entries, 0 to 2499
dtypes: float64(2500)
memory usage: 28.6 MB
None

Sample:       0        1        2        3     ...     2496     2497     2498     2499
0  6.53771  7.04273  7.30593  7.04480  ...  8.13150  8.13992  7.36155  7.98930
1  4.26878  4.05945  4.40541  4.73575  ...  6.57285  8.38097  6.80756  7.12181
2  7.24802  5.96468  7.02855  6.52784  ...  7.23766  6.75104  5.72958  6.73456
3  3.00092  3.33087  3.57794  3.31246  ...  2.81599  2.74684  2.93389  2.76753
4  4.34096  3.79832  5.67286  4.20168  ...  3.26003  2.70133  2.87879  2.64117

[5 rows x 2500 columns]


***********************************
OUTPUT DATA INFO
***********************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  -

In [5]:
# Converting pandas dataframes to numpy arrays for future handling
input_data_np = np.asarray(input_data.values)
output_data_np = np.asarray(output_data.values)
pairs_data_np = np.asarray(pairs_data.values)
print('Input data array sample:\n',input_data_np[:5])
print('\nOutput data array sample:\n',output_data_np[:5])
print('\nPairs data array sample:\n',pairs_data_np[:5])

Input data array sample:
 [[6.53771 7.04273 7.30593 ... 8.13992 7.36155 7.9893 ]
 [4.26878 4.05945 4.40541 ... 8.38097 6.80756 7.12181]
 [7.24802 5.96468 7.02855 ... 6.75104 5.72958 6.73456]
 [3.00092 3.33087 3.57794 ... 2.74684 2.93389 2.76753]
 [4.34096 3.79832 5.67286 ... 2.70133 2.87879 2.64117]]

Output data array sample:
 [[10000.]
 [10000.]
 [10000.]
 [10000.]
 [  270.]]

Pairs data array sample:
 [['D23' 'T194']
 ['D9' 'T270']
 ['D3' 'T47']
 ['D49' 'T222']
 ['D37' 'T28']]


#### Write functions

In [17]:
# Write the functions you need to perform the requested cross-validations.

# Z-standardization using sklearn StandardScaler
def z_standardizer(data_to_scale):
  data_copy = data_to_scale
  scaler = StandardScaler()
  scaled = scaler.fit_transform(data_copy)
  return scaled

# Regression CINDEX
def regression_cindex(true, pred, silent=False):
  n = 0
  n_total = 0
  pairs = list()
  neg_pairs = list()
  pos_pairs = list()
  # Create true,prediction -pairs
  for i in range(len(true)):
    pairs.append([true[i],pred[i]])
  # Pair comparisons
  for i in range(len(pairs)):
    for j in range(i+1,len(pairs)):
      if pairs[i][0] != pairs[j][0]:
        test_one = pairs[i][0]<pairs[j][0]
        test_two = pairs[i][1]<pairs[j][1]
        test_three = pairs[i][1]==pairs[j][1]
        n_total += 1
        if test_one == test_two:
          n += 1
          if test_three:
            n -= 0.5
  # Calculate the final C-index score
  cindx = n / n_total
  if not silent:
    print('CONCORDANCE INDEX RESULTS')
    print('-'*35)
    print('N:',n)
    print('N total:',n_total)
    print('C-INDEX:',cindx)
  return cindx

# Leave-One-Out CV
# obs_type is either 'a','b','c' or 'd'
def LOO_cross_validation(dataset_arr,neighbours_n=20,obs_type='a'):
  folds = dataset_arr.copy()
  predictions = []
  true_labels = []
  knn = KNeighborsRegressor(n_neighbors=neighbours_n)

  # Splitting folds and data -- in this case iterating the dataset
  for i in range(len(folds)):
    process = (i+1)/len(folds)*100
    if process%2 == 0:
      print('Processing type',obs_type.upper(),'LOOCV...',process,'%')
    train_set = folds.copy()
    test_set = folds[i]
    train_set = np.delete(train_set, i, 0)
    pairs_test = test_set[:2]
    
    # For observation type A; performs regular LOO split
    if obs_type == 'a':
      y_train = [arr[2] for arr in train_set]
      x_train = [arr[3:] for arr in train_set]

    # For observation type B; disregards common 1st members from training pool
    if obs_type == 'b':
      temp_train_set = list()
      for arr in train_set:
        if arr[0] != pairs_test[0]:
          temp_train_set.append(arr)
      train_set = np.asarray(temp_train_set)
      y_train = [arr[2] for arr in train_set]
      x_train = [arr[3:] for arr in train_set]
    
    # For observation type C; disregards common 2nd members from training pool
    if obs_type == 'c':
      temp_train_set = list()
      for arr in train_set:
        if arr[1] != pairs_test[1]:
          temp_train_set.append(arr)
      train_set = np.asarray(temp_train_set)
      y_train = [arr[2] for arr in train_set]
      x_train = [arr[3:] for arr in train_set]

    # For observation type D; disregards common 1st and 2nd members from training pool
    if obs_type == 'd':
      temp_train_set = list()
      for arr in train_set:
        first_mem = arr[0] != pairs_test[0]
        second_mem = arr[1] != pairs_test[1]
        if first_mem and second_mem:
          temp_train_set.append(arr)
      train_set = np.asarray(temp_train_set)
      y_train = [arr[2] for arr in train_set]
      x_train = [arr[3:] for arr in train_set]

    knn.fit(x_train, y_train)

    # Test with the index
    y_test = [test_set[2]]
    x_test = [test_set[3:]]
    
    # Gather predictions and true labels
    pred = knn.predict(x_test)
    
    predictions.append(pred[0])
    true_labels.append(y_test)
  print('LOOCV for type',obs_type.upper(),'done!')
  return regression_cindex(true_labels,predictions,True)

In [14]:
# Standardizing inputs and building a collective dataset
z_input = z_standardizer(input_data_np)
collective_data_np = np.concatenate((pairs_data_np,output_data_np,z_input),axis=1)
print('\nCollective pair sample:\n',collective_data_np[:5])


Collective pair sample:
 [['D23' 'T194' 10000.0 ... 0.30551428018223586 0.12096092528634422
  0.25245675687746383]
 ['D9' 'T270' 10000.0 ... 0.3422122349717393 0.04160484646761941
  0.12508837546062354]
 ['D3' 'T47' 10000.0 ... 0.09406829201361683 -0.112809973014698
  0.06823076822172576]
 ['D49' 'T222' 10000.0 ... -0.5155394680698923 -0.5132774832454258
  -0.5142245801189451]
 ['D37' 'T28' 270.0 ... -0.5224680053960499 -0.5211702609903521
  -0.5327772650788085]]


#### Run cross-validations

In [18]:
# Run type A, B, C and D LOOCVs and assign results to variables
ca_result = LOO_cross_validation(collective_data_np,20,'a')
cb_result = LOO_cross_validation(collective_data_np,20,'b')
cc_result = LOO_cross_validation(collective_data_np,20,'c')
cd_result = LOO_cross_validation(collective_data_np,20,'d')

Processing type A LOOCV... 2.0 %
Processing type A LOOCV... 4.0 %
Processing type A LOOCV... 6.0 %
Processing type A LOOCV... 8.0 %
Processing type A LOOCV... 10.0 %
Processing type A LOOCV... 12.0 %
Processing type A LOOCV... 16.0 %
Processing type A LOOCV... 18.0 %
Processing type A LOOCV... 20.0 %
Processing type A LOOCV... 22.0 %
Processing type A LOOCV... 24.0 %
Processing type A LOOCV... 26.0 %
Processing type A LOOCV... 30.0 %
Processing type A LOOCV... 32.0 %
Processing type A LOOCV... 34.0 %
Processing type A LOOCV... 36.0 %
Processing type A LOOCV... 38.0 %
Processing type A LOOCV... 40.0 %
Processing type A LOOCV... 42.0 %
Processing type A LOOCV... 44.0 %
Processing type A LOOCV... 46.0 %
Processing type A LOOCV... 48.0 %
Processing type A LOOCV... 50.0 %
Processing type A LOOCV... 52.0 %
Processing type A LOOCV... 54.0 %
Processing type A LOOCV... 60.0 %
Processing type A LOOCV... 62.0 %
Processing type A LOOCV... 64.0 %
Processing type A LOOCV... 66.0 %
Processing type A 

In [20]:
# Presenting CINDEX results for each case
print('*'*35)
print('Situation 1 performance:',ca_result)
print('*'*35)
print('\n')
print('*'*35)
print('Situation 2 performance:',cc_result)
print('*'*35)
print('\n')
print('*'*35)
print('Situation 3 performance:',cd_result)
print('*'*35)

***********************************
Situation 1 performance: 0.7757329417221724
***********************************


***********************************
Situation 2 performance: 0.7658794108965439
***********************************


***********************************
Situation 3 performance: 0.6762937743725697
***********************************


#### Interpret results

In [ ]:
# Interpret the results you obtained and explain why your cross-validation methods work.

<p><b>Situation 1:</b> The performance estimation has been done by the means of a regular leave-one-out cross-validation (Type A). The dependencies of the remaining pairs are most likely similar to the dataset already in use and thus the model should perform relatively well when predicting their affinities by the means of regular LOOCV.</p>
<p><b>Situation 2:</b> The performance estimation has been done by measuring the success when leaving out the training samples with a common second member (target molecule member) (Type C), since the second member represents the targets this method should well model the described situation task at hand.</p>
<p><b>Situation 3:</b> The performance estimation has been done by leaving out both common pair members (Type D). This way the evaluation should measure how the model handles predicting new, outside the current dataset samples.</p>
<p>Overall, based on the C-index performance measurement where the value of 0.5 would represent a performance no better than a random guess, the model performs best in situation 1, slightly worse in situation 2 and the worst in situation 3.</p>